# Laboratorium 6 - Julia

## Zadanie 1.

In [9]:
function writeOne(n::Integer, c::Channel)
    for i = 1:n
        println("1")
        yield()
    end
end

writeOne (generic function with 2 methods)

In [10]:
function writeTwo(n::Integer, c::Channel)
    for i = 1:n
        println("2")
        yield()
    end
end

writeTwo (generic function with 2 methods)

In [11]:
function writeThree(n::Integer, c::Channel)
    for i = 1:n
        println("3")
        yield()
    end
end

writeThree (generic function with 2 methods)

In [14]:
@sync begin
    c = Channel(32)
    
    @async writeOne(3, c)
    @async writeTwo(3, c)
    @async writeThree(3, c)
end

1
2
3
1
2
3
1
2
3


Task (done) @0x00000001163561d0

In [13]:
@sync begin
    c = Channel(32)
    
    @async writeOne(3, c)
    @async writeTwo(4, c)
    @async writeThree(5, c)
end

1
2
3
1
2
3
1
2
3
2
3
3


Task (done) @0x000000011635f850

## Zadanie 2.

In [1]:
path = "/Users/kubi/Documents/STUDIA/JULIA"
extension = "ipynb"
size = 7

7

In [2]:
function consument(c::Channel)
    localSum = 0
    println("- consument start")
    for file in c
        println("I get ", file)
        localSum += countlines(file)
        yield()
    end
    println("- consument end")
    return localSum
end

consument (generic function with 1 method)

In [5]:
function producer(c::Channel)
    println("+ producent start")
    for(root, dirs, files) in walkdir(path)
    for file in files
        if(extension != "" && split(file, ".")[end] != extension)
            continue
        end
        put!(c, joinpath(root, file))
        yield()
    end
end
    close(C)
    println("+ producent end")
end

producer (generic function with 1 method)

In [7]:
@sync begin
    c = Channel{String}(32)
    sumsArray = SharedArray{Int64}(size)
    @async producer2(c)
    @sync for i = 1:size
        @async sumsArray[i] = consument(c)
    end
    println("The number of lines is: ")
    return sum(sumsArray)
end

- consument start
I get /Users/kubi/Documents/STUDIA/JULIA/lab3_julia.ipynb
- consument start
I get /Users/kubi/Documents/STUDIA/JULIA/lab5_julia.ipynb
- consument start
I get /Users/kubi/Documents/STUDIA/JULIA/.ipynb_checkpoints/lab3_julia-checkpoint.ipynb
- consument start
I get /Users/kubi/Documents/STUDIA/JULIA/.ipynb_checkpoints/lab5_julia-checkpoint.ipynb
- consument start
- consument start
- consument start
I get /Users/kubi/Documents/STUDIA/JULIA/lab4_julia.ipynb
I get /Users/kubi/Documents/STUDIA/JULIA/lab6_julia.ipynb
I get /Users/kubi/Documents/STUDIA/JULIA/.ipynb_checkpoints/lab4_julia-checkpoint.ipynb
I get /Users/kubi/Documents/STUDIA/JULIA/kzajac.github.io/julia/lab1/Tutorial_1.ipynb
I get /Users/kubi/Documents/STUDIA/JULIA/kzajac.github.io/julia/lab3/performance_short_old.ipynb
I get /Users/kubi/Documents/STUDIA/JULIA/kzajac.github.io/julia/lab4/DataAnal_short.ipynb
I get /Users/kubi/Documents/STUDIA/JULIA/kzajac.github.io/julia/lab5/ZadanieAutoDiff.ipynb
I get /Users/k

168213

## Zadanie 3.

In [9]:
using DataFrames
using StatPlots
using Plots
Plots.gr()

LoadError: [91mArgumentError: Module StatPlots not found in current path.
Run `Pkg.add("StatPlots")` to install the StatPlots package.[39m

In [10]:
@everywhere function generate_julia(z; c=2, maxiter=200)
    for i=1:maxiter
        if abs(z) > 2
            return i-1
        end
        z = z^2 + c
    end
    maxiter
end

In [11]:
@everywhere function calc_julia!(julia_set, xrange, yrange; maxiter=200, height=400, width_start=1, width_end=400)
   for x=width_start:width_end
        for y=1:height
            z = xrange[x] + 1im*yrange[y]
            julia_set[x, y] = generate_julia(z, c=-0.70176-0.3842im, maxiter=maxiter)
        end
    end
end

In [12]:
function calc_julia_main(h,w)
   xmin, xmax = -2,2
   ymin, ymax = -1,1
   xrange = linspace(xmin, xmax, w)
   yrange = linspace(ymin, ymax, h)
	println(xrange[100],yrange[101])
   julia_set = Array(Int64, (w, h))
   @time calc_julia!(julia_set, xrange, yrange, height=h, width_end=w)

   Plots.heatmap(xrange, yrange, julia_set)
   png("julia")
end

calc_julia_main (generic function with 1 method)

In [13]:
calc_julia_main(2000,2000)

-1.8019009504752377-0.8999499749874937


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Int64}, ::Tuple{Int64,Int64}) at ./deprecated.jl:57
 [3] calc_julia_main(::Int64, ::Int64) at ./In[12]:7
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/kubi/.julia/v0.6/IJulia/src/execute_request.jl:158
 [6] (::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/kubi/.julia/v0.6/Compat/src/Compat.jl:378
 [7] eventloop(::ZMQ.Socket) at /Users/kubi/.julia/v0.6/IJulia/src/eventloop.jl:8
 [8] (::IJulia.##14#17)() at ./task.jl:335
while loading In[13], in expression starting on line 1


  2.013583 seconds (12.02 M allocations: 672.409 MiB, 3.38% gc time)


LoadError: [91mUndefVarError: Plots not defined[39m